# Opzet eigen Large Language Model met Retrieval Augmented Generation  - LLM met RAG

Vandaag maken we een eigen chatbot mét kennis van lokale documenten. Hieronder staat een stappenplan om deze zelf te bouwen. Maak waar nodig gebruik van een al bestaande chatbot (chatGPT, Mistral, ...) om je code op punt te stellen. Als alles goed is heb je deze daarna (minder hard) nodig...

### Setup

- Creeer een lokale folder met 'local-rag'
- Creëer een virtuele environment, activeer deze
- Installeer chromaDB
- installeer langchain tools, die heb je nodig om je text te splitten
- installeer ollama
- haal het Mistral model binnen - dit kan wel even duren. In de tussentijd kan je best al even verder kijken wat je nog moet doen.
- haal het model binnen om tekst te embedden: nomic-embed-text
- Laat ollama draaien met `ollama serve`

### Webserver
Zet een (Flask of fastapi of andere) webserver op, die volgende routes ondersteunt:
- op de /embed, POST: hier ga je, gebruik makend van een `embed` methode uit de `embed.py` file (zie verder) een document inbedden. Als dit is gelukt geef je een positieve response message terug. Voorzie basic error handling.
- /query, POST: gebruik makend van de `query` methode uit de `query.py`, ga je hier een vraag inlezen.

`jsonify, request, Flask` kunnen nuttige libraries zijn. In essentie is deze webserver gewoon een doorgeefluik voor de volgende codefiles.


### Embedding
**Embedding** in `embed.py` file: voorzie volgende methodes:
- gebruik deze code snippet om enkel pdf's toe te staan:
```python
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in {'pdf'}
```

- Gebruik deze code snippet om een geüploade file op te slagen in een tijdelijke folder:
```python
def save_file(file):
    # Save the uploaded file with a secure filename and return the file path
    ct = datetime.now()
    ts = ct.timestamp()
    filename = str(ts) + "_" + secure_filename(file.filename)
    file_path = os.path.join(TEMP_FOLDER, filename)
    file.save(file_path)

    return file_path
```

- maak een methode `load_and_split_date(file_path)` die een datafile achtereenvolgens
    - inlaadt met `UnstructuredPDFLoader`
    - in stukken splits met een `RecursiveCharacterTextSplitter` - experimenteer met de chunk size en chunk_overlap. Dit bepaalt hoe groot de passages tekst zijn die in de database zullen worden opgeslagen.
    - geef die chunks terug als return van deze methode.

-  een methode `embed(file)`:  Check if the file is valid, save it, load and split the data, add to the database, and remove the temporary file
    - check of de filename geldig is
    - save de file
    - laad deze in en split de data in chunks (met `load_and_split_data`)
    - sla de data op in de vectordatabase (zie verder)
    - delete de tijdelijke data

### Vector database

gebruik deze code voor de vector database. Ze connecteert naadloos met de andere systemen:

```python
import os
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores.chroma import Chroma

CHROMA_PATH = os.getenv('CHROMA_PATH', 'chroma')
COLLECTION_NAME = os.getenv('COLLECTION_NAME', 'local-rag')
TEXT_EMBEDDING_MODEL = os.getenv('TEXT_EMBEDDING_MODEL', 'nomic-embed-text')

def get_vector_db():
    embedding = OllamaEmbeddings(model=TEXT_EMBEDDING_MODEL,show_progress=True)

    db = Chroma(
        collection_name=COLLECTION_NAME,
        persist_directory=CHROMA_PATH,
        embedding_function=embedding
    )

    return db
```

### Query
Tot slot heb je code nodig die op basis van een query extra context gaat zoeken, in de vector database, en deze informatie als context toevoegt aan een query. Implementeer dit op deze manier:
```python
import os
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from get_vector_db import get_vector_db

LLM_MODEL = os.getenv('LLM_MODEL', 'mistral')

def get_prompt():
    QUERY_PROMPT = PromptTemplate(
        input_variables=["question"],
        template="""Hier kan je infor meegeven over hoe de chatbot zich moet gedragen {question}""",
    )

    template = """Answer the question based ONLY on the following context:
    {context}
    Question: {question}
    """

    prompt = ChatPromptTemplate.from_template(template)

    return QUERY_PROMPT, prompt

# Main function to handle the query process
def query(input):
    if input:
        # Initialize the language model with the specified model name
        llm = ChatOllama(model=LLM_MODEL)
        # Get the vector database instance
        db = get_vector_db()
        # Get the prompt templates
        QUERY_PROMPT, prompt = get_prompt()

        # Set up the retriever to generate multiple queries using the language model and the query prompt
        retriever = MultiQueryRetriever.from_llm(
            db.as_retriever(), 
            llm,
            prompt=QUERY_PROMPT
        )

        # Define the processing chain to retrieve context, generate the answer, and parse the output
        chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )

        response = chain.invoke(input)

        return response

    return None

```

Wat nog ontbreekt is de inhoud van de `query(input)` prompt. Ga als volgt tewerk:
- initialiseer een LLM met de geimporteerde chatOllama methode
- gebruik de get_vector_db (hierboven gedefinieerd) om die te kunnen aanroepen
- gebruik de prompt template
- maak een `retriever`, gebruik de `MultiQueryRetriever.from_llm` methode, geef de llm, de database en de prompt mee
- maak een `chain` om alle stukken samen te brengen:
    - context halen met de retriever
    - prompt gebruiken
    - llm aanroepen
    - output parsen met StrOutputParser
- uiteindelijk kan je met `response = chain.invoke(input) ` en `return response` je antwoord teruggeven.

## oplossing

- `pip install --q chromadb`
- `pip install --q unstructured langchain langchain-text-splitters`
- `pip install --q "unstructured[all-docs]`
- `pip install --q langchain_community`
- `pip install --q flask`

Ollama installation:
- zie `setup-ollama.sh` en `install_ollama.sh`

Maak een webserver via een flask:`

```python
import os
from dotenv import load_dotenv

load_dotenv()

from flask import Flask, request, jsonify
from embed import embed
from query import query
from get_vector_db import get_vector_db

TEMP_FOLDER = os.getenv('TEMP_FOLDER', './_temp')
os.makedirs(TEMP_FOLDER, exist_ok=True)

app = Flask(__name__)

@app.route('/embed', methods=['POST'])
def route_embed():
    if 'file' not in request.files:
        return jsonify({"error": "No file part"}), 400

    file = request.files['file']

    if file.filename == '':
        return jsonify({"error": "No selected file"}), 400

    embedded = embed(file)

    if embedded:
        return jsonify({"message": "File embedded successfully"}), 200

    return jsonify({"error": "File embedded unsuccessfully"}), 400

@app.route('/query', methods=['POST'])
def route_query():
    data = request.get_json()
    response = query(data.get('query'))

    if response:
        return jsonify({"message": response}), 200

    return jsonify({"error": "Something went wrong"}), 400

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=8080, debug=True)


```

De embedding heeft als doel om bestanden naar de vector database te brengen zodat hier efficiënt kan worden opgezocht. De steek je in een file `embed.py`:

In [ ]:


import os
from datetime import datetime
from werkzeug.utils import secure_filename
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from get_vector_db import get_vector_db

TEMP_FOLDER = os.getenv('TEMP_FOLDER', './_temp')

# Function to check if the uploaded file is allowed (only PDF files)
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in {'pdf'}

# Function to save the uploaded file to the temporary folder
def save_file(file):
    # Save the uploaded file with a secure filename and return the file path
    ct = datetime.now()
    ts = ct.timestamp()
    filename = str(ts) + "_" + secure_filename(file.filename)
    file_path = os.path.join(TEMP_FOLDER, filename)
    file.save(file_path)

    return file_path

# Function to load and split the data from the PDF file
def load_and_split_data(file_path):
    # Load the PDF file and split the data into chunks
    loader = UnstructuredPDFLoader(file_path=file_path)
    data = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
    chunks = text_splitter.split_documents(data)

    return chunks

# Main function to handle the embedding process
def embed(file):
    # Check if the file is valid, save it, load and split the data, add to the database, and remove the temporary file
    if file.filename != '' and file and allowed_file(file.filename):
        file_path = save_file(file)
        chunks = load_and_split_data(file_path)
        db = get_vector_db()
        db.add_documents(chunks)
        db.persist()
        os.remove(file_path)

        return True

    return False




De query van de gebruiker verloopt via deze code. Let op de globale structuur van de code:
- er wordt een template voorzien met specifieke context
- er wordt een MultiQueryRetriever gebruikt om meerdere antwoorden te genereren. 
- er wordt een antwoord gegeven rekening houdend met de context, die in de vector database te vinden is

`query.py` :

In [ ]:


import os
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from get_vector_db import get_vector_db

LLM_MODEL = os.getenv('LLM_MODEL', 'mistral')

# Function to get the prompt templates for generating alternative questions and answering based on context
def get_prompt():
    QUERY_PROMPT = PromptTemplate(
        input_variables=["question"],
        template="""You are an AI language model assistant. Your task is to generate five
        different versions of the given user question to retrieve relevant documents from
        a vector database. By generating multiple perspectives on the user question, your
        goal is to help the user overcome some of the limitations of the distance-based
        similarity search. Provide these alternative questions separated by newlines.
        Original question: {question}""",
    )

    template = """Answer the question based ONLY on the following context:
    {context}
    Question: {question}
    """

    prompt = ChatPromptTemplate.from_template(template)

    return QUERY_PROMPT, prompt

# Main function to handle the query process
def query(input):
    if input:
        # Initialize the language model with the specified model name
        llm = ChatOllama(model=LLM_MODEL)
        # Get the vector database instance
        db = get_vector_db()
        # Get the prompt templates
        QUERY_PROMPT, prompt = get_prompt()

        # Set up the retriever to generate multiple queries using the language model and the query prompt
        retriever = MultiQueryRetriever.from_llm(
            db.as_retriever(), 
            llm,
            prompt=QUERY_PROMPT
        )

        # Define the processing chain to retrieve context, generate the answer, and parse the output
        chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )

        response = chain.invoke(input)

        return response

    return None





De vector database wordt aanroepen via deze methode:

In [ ]:


import os
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores.chroma import Chroma

CHROMA_PATH = os.getenv('CHROMA_PATH', 'chroma')
COLLECTION_NAME = os.getenv('COLLECTION_NAME', 'local-rag')
TEXT_EMBEDDING_MODEL = os.getenv('TEXT_EMBEDDING_MODEL', 'nomic-embed-text')

def get_vector_db():
    embedding = OllamaEmbeddings(model=TEXT_EMBEDDING_MODEL,show_progress=True)

    db = Chroma(
        collection_name=COLLECTION_NAME,
        persist_directory=CHROMA_PATH,
        embedding_function=embedding
    )

    return db




Als laatste hebben we nog wat environment variabelen nodig in een `.env`

```
TEMP_FOLDER = './_temp'
CHROMA_PATH = 'chroma'
COLLECTION_NAME = 'local-rag'
LLM_MODEL = 'mistral'
TEXT_EMBEDDING_MODEL = 'nomic-embed-text'
```

## Server laten draaien
De server laten draaien doe je met `python app.py`. Nu kan je dan requests sturen, bijvoorbeeld een CV uploaden:

``` 
curl --request POST \
--url http://localhost:8080/embed \
--header 'Content-Type: multipart/form-data' \
--form file=@/PATHTOFILE/CV.pdf
```

Dan kan je vragen stellen over dit CV:

```
curl --request POST \
  --url http://localhost:8080/query \
  --header 'Content-Type: application/json' \
  --data '{ "query": "Wat heeft -- gestudeerd ?" }'
```